## Set up

### Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

### Load data

In [ ]:
path_to_file = "all.txt"
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 314347 characters


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

82 unique characters


## Process the text

### Vectorize the text

In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '&' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '+' :   9,
  ',' :  10,
  '-' :  11,
  '.' :  12,
  '/' :  13,
  '0' :  14,
  '1' :  15,
  '2' :  16,
  '3' :  17,
  '4' :  18,
  '5' :  19,
  ...
}


### Create training examples and targets

In [ ]:
seq_length = 100
examples_per_epoch = len(text)//seq_length

# create training examples and targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

1
)
 
W
h


In [ ]:
# The batch method lets you easily convert these individual characters to sequences of the desired size.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"1) What is the reason for your visit today?\r\nDiscuss getting referral to Brigham & Women's in Mass , "
'just for 2nd opinion.\r\n2) Are you RIGHT or LEFT handed?\r\nRight\r\n3) When did this problem start?\r\nUnkn'
'own\r\n4) Has this problem happened before?\r\nNo\r\n5) How did it start?\r\nNo answer\r\n6) Has it changed sin'
'ce it started?\r\nNo\r\n7) Have you seen another physician for this problem?\r\nNo\r\n8) What work-up has bee'
'n done for this problem?\r\nMRI Brain, Neurology Assoc.\r\n9) Do you have a personal history of cancer or'


In [ ]:
# a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "1) What is the reason for your visit today?\r\nDiscuss getting referral to Brigham & Women's in Mass ,"
Target data: ") What is the reason for your visit today?\r\nDiscuss getting referral to Brigham & Women's in Mass , "


In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 15 ('1')
  expected output: 8 (')')
Step    1
  input: 8 (')')
  expected output: 2 (' ')
Step    2
  input: 2 (' ')
  expected output: 50 ('W')
Step    3
  input: 50 ('W')
  expected output: 61 ('h')
Step    4
  input: 61 ('h')
  expected output: 54 ('a')


### Create training batches

In [ ]:
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements). 
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

In [ ]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

## Try the model

In [ ]:
# check the shape of the output
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 82) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           20992     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 82)            84050     
Total params: 4,043,346
Trainable params: 4,043,346
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Try it for the first example in the batch
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

array([19, 34,  5, 71, 24,  4, 21,  1,  8, 36, 39, 52, 60, 61, 72, 37, 20,
       37, 50, 64, 45, 26, 62, 32, 12, 63, 48, 48, 18, 80, 46, 61, 28,  3,
       80, 20, 30, 61,  7, 51, 43, 68, 28, 66, 38, 50, 22, 12, 60, 34, 71,
       71, 75, 53, 55, 22, 64, 44, 75, 15, 46, 38,  8, 58, 51, 67, 51, 73,
       68, 62, 34, 23, 33, 62, 65, 30, 57, 34, 11, 24, 73, 55, 13, 19, 14,
       24, 30, 18, 29, 71, 37, 15, 78, 10, 17,  1, 73, 35, 59, 70])

In [ ]:
# Decode these to see the text predicted by this untrained model
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "the reason for today's visit is: Surgery follow up. The patient reports that they are Right handed. "

Next Char Predictions: 
 '5G&r:"7\r)ILYghsJ6JWkR?iE.jUU4—ShA!—6Ch(XPoAmKW8.gGrrv_b8kQv1SK)eXnXtoiG9FilCdG-:tb/50:C4BrJ1y,3\rtHfq'


## Train the model

### Attach an optimizer, and a loss function

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 82)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4076633


In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [ ]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    save_freq=500,
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [ ]:
EPOCHS=2000

In [24]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/2000
48/48 [==============================] - 3s 29ms/step - loss: 3.9877
Epoch 2/2000
48/48 [==============================] - 2s 28ms/step - loss: 2.5024
Epoch 3/2000
48/48 [==============================] - 2s 29ms/step - loss: 1.9746
Epoch 4/2000
48/48 [==============================] - 2s 29ms/step - loss: 1.5717
Epoch 5/2000
48/48 [==============================] - 2s 29ms/step - loss: 1.1184
Epoch 6/2000
48/48 [==============================] - 2s 29ms/step - loss: 0.8209
Epoch 7/2000
48/48 [==============================] - 2s 29ms/step - loss: 0.6906
Epoch 8/2000
48/48 [==============================] - 2s 29ms/step - loss: 0.5958
Epoch 9/2000
48/48 [==============================] - 2s 29ms/step - loss: 0.5388
Epoch 10/2000
48/48 [==============================] - 2s 29ms/step - loss: 0.4870
Epoch 11/2000
48/48 [==============================] - 2s 31ms/step - loss: 0.4444
Epoch 12/2000
48/48 [==============================] - 2s 29ms/step - loss: 0.3961
Epoch 13/2000

## Generate text

In [25]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_2000'

In [26]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            20992     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 82)             84050     
Total params: 4,043,346
Trainable params: 4,043,346
Non-trainable params: 0
_________________________________________________________________


In [28]:
def generate_text(model, start_string):

  num_generate = 1000

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [29]:
print(generate_text(model, start_string=u"""
1) What is the reason for your visit today?
Review my MRI scan from 11/10
2) Are you RIGHT or LEFT handed?
Right
3) When did this problem start?
A few years ago
4) Has this problem happened before?
No
5) How did it start?
I was injinjured in a car accaccident 
6) Has it changed since it started?
No
7) Have you seen another physician for this problem?
No
8) What work-up has been done for this problem?
MRI Brain, Neurology Assoc.
9) Do you have a personal history of cancer or tumors?
No
10) If you answered yes to a personal history of cancer or tumors, please relate the details of the diagnosis and prior treatments
No answer
11) Do you have a personal history of any disease or condition relating to the visit today?
No 
12) Review of Systems: Do you have any of the following problems?
Heat Intolerance, Cold Intolerance, Blurred Vision
13) Review of Systems: Do you have any of the following problems?
No answer
14) Review of Systems: Do you have any of the following problems?
Nausea, Back pain, Neck pain, Joint pain
15) Review of Systems: Do you have any of the following problems?
Anxiety, Numbness, Tingling, Weakness
16) Review of Systems: Do you have any of the following problems?
No answer
17) Over the last two weeks how often have you been bothered by any of the following problems?
Score: 10
"""))


1) What is the reason for your visit today?
Review my MRI scan from 11/10
2) Are you RIGHT or LEFT handed?
Right
3) When did this problem start?
A few years ago
4) Has this problem happened before?
No
5) How did it start?
I was injinjured in a car accaccident 
6) Has it changed since it started?
No
7) Have you seen another physician for this problem?
No
8) What work-up has been done for this problem?
MRI Brain, Neurology Assoc.
9) Do you have a personal history of cancer or tumors?
No
10) If you answered yes to a personal history of cancer or tumors, please relate the details of the diagnosis and prior treatments
No answer
11) Do you have a personal history of any disease or condition relating to the visit today?
No 
12) Review of Systems: Do you have any of the following problems?
Heat Intolerance, Cold Intolerance, Blurred Vision
13) Review of Systems: Do you have any of the following problems?
No answer
14) Review of Systems: Do you have any of the following problems?
Nausea, Back 